##### To do:
    - Prevent obstacles from blocking all pathways (Dont put obstacles in the first row)
    - Code GW as an Object

In [1]:
import numpy as np
import copy

In [48]:
# Specify dimensions

while True:    
    try:
        s = int(input("Please provide your desired grid dimension (dim X dim):\n"))
        
        if s >= 5:
            break
            
        print("Dimension needs to be larger than 4!\n")
    
    except:
        print("Please provide an integer value!\n")

# Create basic structure

for x in range(s+1):
    grid_world = ["O"] * (x * x)

# Create obstacles

grid_world = np.array(grid_world)

num_obs = int((x*x)/8) # specify number of obstacles

obstacle_indices = np.random.choice(np.arange(grid_world.size), replace=False,
                           size=num_obs)

grid_world[obstacle_indices] = "X"

# Set starting point

grid_world[0] = "P" # top left

# Show GW

grid_world = grid_world.reshape(((x, x)))

# Save Backup

grid_world_original = copy.deepcopy(grid_world)

print("Gridworld:\n")
print(grid_world)
print("\n")

Please provide your desired grid dimension (dim X dim):
15
Gridworld:

[['P' 'O' 'O' 'O' 'O' 'O' 'O' 'O' 'O' 'O' 'O' 'O' 'O' 'O' 'O']
 ['O' 'O' 'O' 'O' 'O' 'O' 'O' 'X' 'O' 'X' 'O' 'O' 'O' 'O' 'O']
 ['O' 'O' 'O' 'O' 'O' 'O' 'O' 'O' 'O' 'O' 'O' 'O' 'O' 'O' 'O']
 ['O' 'O' 'O' 'O' 'X' 'X' 'X' 'O' 'O' 'X' 'O' 'O' 'O' 'O' 'O']
 ['O' 'O' 'O' 'O' 'O' 'O' 'X' 'X' 'X' 'O' 'O' 'O' 'O' 'O' 'O']
 ['O' 'O' 'O' 'O' 'O' 'O' 'O' 'O' 'X' 'O' 'O' 'O' 'X' 'X' 'O']
 ['O' 'O' 'O' 'O' 'O' 'O' 'X' 'O' 'O' 'O' 'X' 'O' 'O' 'O' 'O']
 ['O' 'X' 'O' 'O' 'O' 'X' 'O' 'O' 'O' 'O' 'O' 'O' 'O' 'O' 'O']
 ['O' 'O' 'O' 'O' 'O' 'O' 'O' 'O' 'O' 'O' 'O' 'O' 'O' 'O' 'O']
 ['O' 'O' 'O' 'X' 'O' 'O' 'O' 'O' 'O' 'O' 'O' 'O' 'O' 'O' 'O']
 ['O' 'O' 'O' 'O' 'O' 'O' 'O' 'O' 'O' 'O' 'O' 'O' 'O' 'O' 'O']
 ['O' 'O' 'O' 'X' 'O' 'X' 'X' 'X' 'O' 'O' 'O' 'O' 'O' 'O' 'O']
 ['O' 'O' 'X' 'O' 'O' 'O' 'O' 'O' 'O' 'X' 'O' 'X' 'O' 'X' 'O']
 ['O' 'O' 'O' 'X' 'O' 'O' 'O' 'O' 'O' 'O' 'O' 'O' 'O' 'O' 'O']
 ['O' 'X' 'O' 'X' 'O' 'O' 'O' 'O' 'O' 'O' 'O' '

In [49]:
# Assign rewards to states

reward_world =copy.deepcopy(grid_world.flatten()) # create seperate array for reward

rew_neg_amount = 5 # specify number of negative rewards

free_fields = [x for x in np.arange(reward_world.size) if x not in obstacle_indices] # generate list of free fields

np.random.shuffle(free_fields) # shuffle free fields

rew_pos_index = free_fields[-1] # choose index for postive reward (terminal state)

rew_neg_indices = np.random.choice(free_fields[0:-1], replace=False, size=rew_neg) # randomly choose incides for negative rewards

reward_world[rew_neg_indices] = "-" # place negative rewards

reward_world[rew_pos_index] = "+" # place positive reward (terminal state)

reward_world = reward_world.reshape(((x, x)))

# Save Backup

reward_world_original = copy.deepcopy(reward_world)

# Show

print("Reward world:\n")
print(reward_world)

Reward world:

[['P' 'O' 'O' 'O' 'O' 'O' 'O' 'O' 'O' 'O' 'O' 'O' 'O' 'O' 'O']
 ['-' 'O' 'O' 'O' 'O' 'O' 'O' 'X' 'O' 'X' 'O' 'O' 'O' 'O' 'O']
 ['O' 'O' 'O' 'O' 'O' 'O' 'O' 'O' 'O' 'O' 'O' 'O' 'O' 'O' 'O']
 ['O' 'O' 'O' 'O' 'X' 'X' 'X' 'O' 'O' 'X' 'O' 'O' 'O' 'O' 'O']
 ['O' 'O' 'O' 'O' 'O' 'O' 'X' 'X' 'X' 'O' 'O' 'O' 'O' 'O' 'O']
 ['O' '+' 'O' 'O' 'O' 'O' 'O' 'O' 'X' 'O' 'O' 'O' 'X' 'X' 'O']
 ['O' 'O' 'O' 'O' '-' 'O' 'X' 'O' 'O' 'O' 'X' 'O' 'O' 'O' 'O']
 ['O' 'X' 'O' 'O' 'O' 'X' 'O' 'O' 'O' 'O' 'O' 'O' 'O' 'O' 'O']
 ['O' 'O' 'O' 'O' 'O' 'O' 'O' 'O' 'O' 'O' 'O' 'O' 'O' 'O' 'O']
 ['O' 'O' 'O' 'X' 'O' 'O' 'O' 'O' 'O' 'O' 'O' 'O' 'O' 'O' '-']
 ['O' 'O' 'O' 'O' 'O' 'O' 'O' 'O' 'O' 'O' 'O' 'O' 'O' 'O' 'O']
 ['O' 'O' 'O' 'X' 'O' 'X' 'X' 'X' 'O' 'O' 'O' 'O' 'O' 'O' 'O']
 ['-' 'O' 'X' 'O' 'O' 'O' 'O' 'O' 'O' 'X' 'O' 'X' 'O' 'X' 'O']
 ['O' 'O' 'O' 'X' 'O' 'O' 'O' 'O' 'O' 'O' 'O' 'O' 'O' 'O' 'O']
 ['O' 'X' 'O' 'X' 'O' 'O' 'O' '-' 'O' 'O' 'O' 'O' 'O' 'O' 'O']]


In [ ]:
def reset_gridworld(grid_world,reward_world):
    reward_world = reward_world_original
    grid_world = grid_world_original
    
    print("Reset successfull!\n")
    
    return reward_world, grid_world # is this necessary after assigning values earlier?

def step(grid_world,reward_world):
    